In [1]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException

In [ ]:
# Update with your chromedriver path
service = ChromeService(executable_path='/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=service)

driver.get("""https://www.blocket.se/bilar/sok?filter=%7B"key"%3A"modelYear"%2C"range"%3A%7B"start"%3A"2015"%2C"end"%3A""%7D%7D&filter=%7B"key"%3A"sellerType"%2C"values"%3A%5B"Privat"%5D%7D""")

try:
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "#sp_message_iframe_1253915"))
    )
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sp_choice_type_11"))
    ).click()
finally:
    driver.switch_to.default_content()

input("Press Enter to quit...")
driver.quit()

In [7]:
import os
import re
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

START_URL = (
    "https://www.blocket.se/bilar/sok?"
    "filter=%7B\"key\"%3A\"modelYear\"%2C\"range\"%3A%7B\"start\"%3A\"2015\"%2C\"end\"%3A\"\"%7D%7D&"
    "filter=%7B\"key\"%3A\"sellerType\"%2C\"values\"%3A%5B\"Privat\"%5D%7D"
)

CARS_FOLDER = "cars"
LINKS_FILE = "car_links.txt"

# Path to your ChromeDriver
service = ChromeService(executable_path="/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)

def bypass_cookies():
    """Try to click away any cookie banner (if present)."""
    try:
        WebDriverWait(driver, 5).until(
            EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe[id^='sp_message_iframe_']"))
        )
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sp_choice_type_11"))
        ).click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()

def sanitize_filename(s):
    """Clean up strings so they can be used as filenames."""
    return re.sub(r'[\\/:*?"<>|]+', '_', s)

def collect_ads_on_page():
    """
    Collects all 'annons' links on the current page.
    """
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.list.w-full > div > a"))
    )
    anchors = driver.find_elements(By.CSS_SELECTOR, "div.list.w-full > div > a")
    links = []
    for a in anchors:
        href = a.get_attribute("href")
        if href and "annons" in href:
            links.append(href)
    return links

def click_next_page():
    """
    Attempts to click the 'Nästa' button (or anchor).  
    Returns True if a next page is found and clicked, otherwise False.
    """
    try:
        # Adjust the selector as needed for Blocket. 
        # This example uses your given CSS path for the next-page button.
        next_btn = driver.find_element(
            By.CSS_SELECTOR,
            "#__next > main > div.container.mx-auto.mt-3.space-y-4.md\\:mt-6 "
            "> div > div.flex.flex-col.w-full.space-y-6.lg\\:col-span-3.md\\:items-center "
            "> div:nth-child(3) > div > div > div:nth-child(2) "
            "> div.pagination.flex.w-full.justify-between.gap-3.items-stretch.min-h-\\[44px\\] "
            "> div.flex.justify-end.flex-1 > a > button"
        )
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)  # Give time for next page to load
        return True
    except:
        return False

def gather_all_links():
    """
    1) Go to START_URL
    2) Collect all ad links from each page
    3) Paginate until no 'Nästa' is found
    4) Writes them into LINKS_FILE
    """
    driver.get(START_URL)
    bypass_cookies()

    all_links = []
    while True:
        page_links = collect_ads_on_page()
        all_links.extend(page_links)
        if not click_next_page():
            break
    
    # Write them to a text file
    with open(LINKS_FILE, "w", encoding="utf-8") as f:
        for link in all_links:
            f.write(link + "\n")
    print(f"[INFO] Wrote {len(all_links)} links to {LINKS_FILE}.")

def extract_price():
    """Returns only digits (numbers) from the price element, or 'N/A' if not found."""
    try:
        el = driver.find_element(By.CSS_SELECTOR, "div[class*='Price__StyledPrice']")
        return re.sub(r"\D", "", el.text.strip()) or "N/A"
    except:
        return "N/A"

def get_ad_details():
    """Wait for the article to appear, then get the title + price."""
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "article"))
    )
    try:
        title = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
    except:
        title = "UnknownTitle"
    return title, extract_price()

def save_carousel_images(folder):
    """
    Loop through the image carousel, collect image URLs, and save them.
    """
    CSS_DIVS = "article div[style*='background-image']"
    CSS_NEXT_BTN = "button.SliderControls__StyledButton-sc-1dbsnpt-4.cIKvvT"
    all_urls, prev_count = set(), 0

    for _ in range(25):  # Arbitrary max slides
        divs = driver.find_elements(By.CSS_SELECTOR, CSS_DIVS)
        for d in divs:
            style = d.get_attribute("style")
            m = re.search(r'url\("([^"]+)"\)', style)
            if m:
                all_urls.add(m.group(1))
        if len(all_urls) == prev_count:
            break
        prev_count = len(all_urls)
        try:
            btn = driver.find_element(By.CSS_SELECTOR, CSS_NEXT_BTN)
            if btn.is_enabled() and btn.is_displayed():
                btn.click()
                time.sleep(1)
            else:
                break
        except:
            break

    # Now save them to disk
    for i, url in enumerate(all_urls, 1):
        img_path = os.path.join(folder, f"img_{i}.jpg")
        try:
            with open(img_path, "wb") as f:
                f.write(requests.get(url).content)
        except Exception as e:
            print(f"Failed to save image from {url}: {e}")

def save_parameters(folder):
    """
    Save label-value pairs of ad parameters (e.g. brand, model year, etc.) to separate text files.
    """
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[class*='ExpandableContent__Content']"))
        )
        labels = driver.find_elements(By.CSS_SELECTOR, "div[class*='ParamsWithIcons__StyledLabel']")
        values = driver.find_elements(By.CSS_SELECTOR, "div[class*='ParamsWithIcons__StyledParamValue']")
        for i in range(min(len(labels), len(values))):
            lbl = labels[i].text.strip() or f"NoLabel_{i}"
            val = values[i].text.strip() or "N/A"
            fn = sanitize_filename(lbl)
            with open(os.path.join(folder, f"{fn}.txt"), "w", encoding="utf-8") as ff:
                ff.write(val)
    except:
        pass

def scrape_single_ad(url):
    """
    Load the ad page, gather info, save everything to disk in CARS_FOLDER.
    """
    driver.get(url)
    bypass_cookies()

    title, price = get_ad_details()

    # Make a folder for this particular ad (limit title length to 80 chars)
    folder_name = sanitize_filename(title)[:80] or "NoTitle"
    folder = os.path.join(CARS_FOLDER, folder_name)
    os.makedirs(folder, exist_ok=True)

    # Save basic info
    with open(os.path.join(folder, "price.txt"), "w", encoding="utf-8") as f:
        f.write(price)
    with open(os.path.join(folder, "url.txt"), "w", encoding="utf-8") as f:
        f.write(url)

    # Save images
    save_carousel_images(folder)
    # Save label-value parameters
    save_parameters(folder)

def scrape_from_file():
    """
    Reads the links from LINKS_FILE and scrapes each ad.
    Skips any ad that already has a 'url.txt' saved in its folder.
    """
    if not os.path.exists(LINKS_FILE):
        print(f"[ERROR] File {LINKS_FILE} not found. Run gather_all_links() first.")
        return

    with open(LINKS_FILE, "r", encoding="utf-8") as f:
        all_links = [line.strip() for line in f if line.strip()]

    print(f"[INFO] Found {len(all_links)} links in {LINKS_FILE}. Starting scrape...")

    os.makedirs(CARS_FOLDER, exist_ok=True)

    skipped = 0
    scraped = 0

    for link in all_links:
        try:
            # Recreate folder name from ad title preview
            driver.get(link)
            bypass_cookies()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1"))
            )
            title = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
            folder_name = sanitize_filename(title)[:80] or "NoTitle"
            folder = os.path.join(CARS_FOLDER, folder_name)

            # Skip if folder with url.txt already exists
            if os.path.exists(os.path.join(folder, "url.txt")):
                skipped += 1
                print(f"[SKIP] Already scraped: {folder}")
                continue

            scrape_single_ad(link)
            scraped += 1

        except Exception as e:
            print(f"[ERROR] {link} -> {e}")
            continue

    print(f"[DONE] Scraped {scraped} new ads, skipped {skipped} existing ones.")


def main():
    # Step 1: Gather all links (uncomment if you want to always re-collect them):
    gather_all_links()
    # Close or refresh the browser if needed (but let's just reuse the same driver)
    
    # Step 2: Scrape based on the links file
    scrape_from_file()

    # Done
    driver.quit()

if __name__ == "__main__":
    main()

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=125.0.6422.141); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000104e2a4c8 chromedriver + 4302024
1   chromedriver                        0x0000000104e22e10 chromedriver + 4271632
2   chromedriver                        0x0000000104a5419c chromedriver + 278940
3   chromedriver                        0x0000000104a5866c chromedriver + 296556
4   chromedriver                        0x0000000104a5a740 chromedriver + 304960
5   chromedriver                        0x0000000104acf72c chromedriver + 784172
6   chromedriver                        0x0000000104acec5c chromedriver + 781404
7   chromedriver                        0x0000000104a8b004 chromedriver + 503812
8   chromedriver                 

In [ ]:
import os
import re
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

START_URL = (
    "https://www.blocket.se/bilar/sok?"
    "filter=%7B\"key\"%3A\"modelYear\"%2C\"range\"%3A%7B\"start\"%3A\"2015\"%2C\"end\"%3A\"\"%7D%7D&"
    "filter=%7B\"key\"%3A\"sellerType\"%2C\"values\"%3A%5B\"Privat\"%5D%7D"
)

CARS_FOLDER = "cars"
LINKS_FILE = "car_links.txt"

service = ChromeService(executable_path="/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)

def bypass_cookies():
    """Try to click away any cookie banner (if present)."""
    try:
        WebDriverWait(driver, 5).until(
            EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe[id^='sp_message_iframe_']"))
        )
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sp_choice_type_11"))
        ).click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()

def sanitize_filename(s):
    """Clean up strings so they can be used as filenames."""
    return re.sub(r'[\\/:*?"<>|]+', '_', s)

def extract_price():
    """Returns only digits from the price element, or 'N/A' if not found."""
    try:
        el = driver.find_element(By.CSS_SELECTOR, "div[class*='Price__StyledPrice']")
        return re.sub(r"\D", "", el.text.strip()) or "N/A"
    except:
        return "N/A"

def get_ad_details():
    """Wait for the article to appear, then get the title and price."""
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "article"))
    )
    try:
        title = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
    except:
        title = "UnknownTitle"
    return title, extract_price()

def save_carousel_images(folder):
    """Loop through the image carousel, collect image URLs, and save them."""
    CSS_DIVS = "article div[style*='background-image']"
    CSS_NEXT_BTN = "button.SliderControls__StyledButton-sc-1dbsnpt-4.cIKvvT"
    all_urls, prev_count = set(), 0

    for _ in range(25):  # arbitrary max slides
        divs = driver.find_elements(By.CSS_SELECTOR, CSS_DIVS)
        for d in divs:
            style = d.get_attribute("style")
            m = re.search(r'url\("([^"]+)"\)', style)
            if m:
                all_urls.add(m.group(1))
        if len(all_urls) == prev_count:
            break
        prev_count = len(all_urls)
        try:
            btn = driver.find_element(By.CSS_SELECTOR, CSS_NEXT_BTN)
            if btn.is_enabled() and btn.is_displayed():
                btn.click()
                time.sleep(1)
            else:
                break
        except:
            break

    # Save them to disk
    for i, url in enumerate(all_urls, 1):
        img_path = os.path.join(folder, f"img_{i}.jpg")
        try:
            with open(img_path, "wb") as f:
                f.write(requests.get(url).content)
        except Exception as e:
            print(f"Failed to save image from {url}: {e}")

def save_parameters(folder):
    """Save label-value pairs of ad parameters to separate text files."""
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[class*='ExpandableContent__Content']"))
        )
        labels = driver.find_elements(By.CSS_SELECTOR, "div[class*='ParamsWithIcons__StyledLabel']")
        values = driver.find_elements(By.CSS_SELECTOR, "div[class*='ParamsWithIcons__StyledParamValue']")
        for i in range(min(len(labels), len(values))):
            lbl = labels[i].text.strip() or f"NoLabel_{i}"
            val = values[i].text.strip() or "N/A"
            fn = sanitize_filename(lbl)
            with open(os.path.join(folder, f"{fn}.txt"), "w", encoding="utf-8") as ff:
                ff.write(val)
    except:
        pass

def scrape_single_ad(url):
    """
    Load the ad page, gather info, save everything to disk in CARS_FOLDER.
    """
    driver.get(url)
    bypass_cookies()

    title, price = get_ad_details()

    folder_name = sanitize_filename(title)[:80] or "NoTitle"
    folder = os.path.join(CARS_FOLDER, folder_name)
    os.makedirs(folder, exist_ok=True)

    # Save basic info
    with open(os.path.join(folder, "price.txt"), "w", encoding="utf-8") as f:
        f.write(price)
    with open(os.path.join(folder, "url.txt"), "w", encoding="utf-8") as f:
        f.write(url)

    # Save images
    save_carousel_images(folder)
    # Save label-value parameters
    save_parameters(folder)

def scrape_from_file():
    """
    Reads the links from LINKS_FILE and scrapes each ad, skipping any ad that's
    already stored (i.e., the folder with url.txt already exists).
    """
    if not os.path.exists(LINKS_FILE):
        print(f"[ERROR] File {LINKS_FILE} not found. Please provide a valid file.")
        return

    with open(LINKS_FILE, "r", encoding="utf-8") as f:
        all_links = [line.strip() for line in f if line.strip()]

    print(f"[INFO] Found {len(all_links)} links in {LINKS_FILE}. Starting scrape...")
    os.makedirs(CARS_FOLDER, exist_ok=True)

    skipped = 0
    scraped = 0

    for link in all_links:
        try:
            # Peek at the ad title (to know folder name) before deciding if we skip
            driver.get(link)
            bypass_cookies()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1"))
            )
            title = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
            folder_name = sanitize_filename(title)[:80] or "NoTitle"
            folder = os.path.join(CARS_FOLDER, folder_name)

            # Skip if folder with url.txt already exists
            if os.path.exists(os.path.join(folder, "url.txt")):
                skipped += 1
                print(f"[SKIP] Already scraped: {folder}")
                continue

            # Otherwise, do the full scrape
            scrape_single_ad(link)
            scraped += 1

        except Exception as e:
            print(f"[ERROR] {link} -> {e}")
            continue

    print(f"[DONE] Scraped {scraped} new ads, skipped {skipped} existing ones.")

def main():
    # Instead of gathering links, we directly call scrape_from_file.
    scrape_from_file()
    driver.quit()

if __name__ == "__main__":
    main()


[INFO] Found 6111 links in car_links.txt. Starting scrape...
[SKIP] Already scraped: cars/Toyota Prius+ Hybrid CVT Euro 6
[SKIP] Already scraped: cars/Audi A6 Avant 2.0 TDI clean diesel quattro S Tronic Ambition
[SKIP] Already scraped: cars/Nissan Qashqai 1.5 dCi Euro 6
[SKIP] Already scraped: cars/Volkswagen ID.3 Max
[SKIP] Already scraped: cars/Chevrolet Camaro 3.6 V6 Hydra-Matic
[SKIP] Already scraped: cars/Tesla Model 3


In [6]:
import shutil
import os

def remove_dirs_with_slap(cars_folder="cars"):
    removed_count = 0
    total_count = 0
    for root, dirs, files in os.walk(cars_folder):
        if "Biltyp.txt" in files:
            total_count += 1
            with open(os.path.join(root, "Biltyp.txt"), "r", encoding="utf-8") as f:
                content = f.read()
                if "Släp" in content or "Slap" in content or "Husvagn" in content or "Vagn" in content or "Husbil" in content:
                    print(f"[INFO] Removing directory: {root}")
                    shutil.rmtree(root)
                    removed_count += 1
    print(f"[INFO] Removed {removed_count} directories. {total_count - removed_count} directories left.")

remove_dirs_with_slap()


[INFO] Removed 0 directories. 813 directories left.


In [1]:
import os

def create_already_matched_file(cars_folder="cars"):
    already_matched_links = []
    for root, dirs, files in os.walk(cars_folder):
        if "url.txt" in files:
            with open(os.path.join(root, "url.txt"), "r", encoding="utf-8") as f:
                links = f.readlines()
                already_matched_links.extend([link.strip() for link in links])

    with open("already_matched.txt", "w", encoding="utf-8") as f:
        for link in already_matched_links:
            f.write(f"{link}\n")

create_already_matched_file()
